In [1]:
import sys
import os
import gensim
sys.path.append("../..")  # adding Recommenders root path.
print(sys.path)

['/opt/caffe/python', '/opt/caffe2/build', '/data/home/wutao/src/Recommenders/notebooks/00_quick_start', '/data/anaconda/envs/reco_base/lib/python36.zip', '/data/anaconda/envs/reco_base/lib/python3.6', '/data/anaconda/envs/reco_base/lib/python3.6/lib-dynload', '', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages/IPython/extensions', '/data/home/wutao/.ipython', '/data/anaconda/envs/reco_base/lib/python3.6/site-packages/azureml/_project/vendor', '../..']


In [2]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

import logging
import time

import numpy as np
import pandas as pd
#import papermill as pm
import gensim
import multiprocessing

from numpy import random
#from sklearn.cluster import KMeans

from reco_utils.dataset import movielens
from reco_utils.dataset.movielens import load_pandas_df
from reco_utils.dataset.python_splitters import python_stratified_split,python_chrono_split
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, 
    precision_at_k, recall_at_k, get_top_k_items)


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))


System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Pandas version: 0.24.2


In [3]:
data = load_pandas_df(size='100k', genres_col='genre', title_col='title', year_col='year')

# Convert the float precision to 32-bit in order to reduce memory consumption 
data['rating'] = data['rating'].astype(np.float32)
data[['userID','itemID']] = data[['userID','itemID']].astype(str)


100%|██████████| 4.81k/4.81k [00:00<00:00, 11.8kKB/s]


In [4]:
#train, test = python_stratified_split(data, ratio=0.75, col_user='userID', col_item='itemID', seed=42)
train, test = python_chrono_split(data, ratio=0.75, col_user='userID', col_item='itemID')
#train, validation, test = python_chrono_split(data, ratio=[0.7, 0.05, 0.25])
train_dict = dict(train.groupby('userID')['itemID'].apply(list))
test_dict = dict(test.groupby('userID')['itemID'].apply(list))
train_users_list = list(train_dict.keys()) 

In [5]:
train_distribution_list = [0] * len(train_users_list)

for i in range(len(train_users_list)):
    train_distribution_list[i] = len(train_dict[train_users_list[i]])

total_viewings = sum(train_distribution_list)
prob_train = [user_num_movies / total_viewings for user_num_movies in train_distribution_list]

In [6]:
ITER = 50000
WINDOW = 10+1
REP = 50
PROPORTION_USAGE = 0.80
ADD_UID = False
DIM=20

TOP_K = 10
TOP_K_MULTIPLIER = 1

TOPN_PERCLUSTER = 20
#MAX_CLUSTERS = 15

random.seed(2)

EXP_NAME = "chrono_50k_080_20dim_075_025"
datafile = EXP_NAME + ".data"
modelfile = EXP_NAME + ".model"

In [7]:
#NO NEED TO RUN IF data file is already available

with open(datafile,"w") as f:
    for i in range(ITER):
        if i / ITER < PROPORTION_USAGE:  # sample based on user's usage.  The more ratings a user has, the higher probability of being selected.
            user_idx = random.choice(len(prob_train),p=prob_train)
        else:  # go through user population sequentially
            user_idx = i % len(train_users_list)
            
        selected_userID = train_users_list[user_idx]
        selected_user_movies_list = train_dict[selected_userID]
        if len(selected_user_movies_list) >= WINDOW:
            window = WINDOW
        else:
            window = len(selected_user_movies_list)
        
        temp_movies_list = []
        uid_string = "u"+str(selected_userID)
        
        for j in range(REP):
# now switching to numpy.random            idxsample = random.sample(range(len(rand_user_movies_list)),window)
            idxsample = random.randint(len(selected_user_movies_list),size=window)
            temp_movies_list = temp_movies_list + [selected_user_movies_list[idx] for idx in idxsample]
            if ADD_UID:
                temp_movies_list.append(uid_string)
                
        f.write(' '.join(temp_movies_list)+'\n')
        

In [8]:
def read_seq():
    with open(datafile,"rt") as f:
        for i, line in enumerate(f):
            yield line.strip().split()

docs = list(read_seq())

In [9]:
start_time = time.time()
model = gensim.models.Word2Vec(docs, size = DIM, window = 5, min_count =2 , 
                               workers = multiprocessing.cpu_count(), sg = 1)
model.train(docs, total_examples= len(docs), epochs = 10, compute_loss = True)

(256529348, 275000000)

In [10]:
train_time = time.time() - start_time
print("Took {} seconds for training.".format(train_time))

Took 168.14429259300232 seconds for training.


In [11]:
model.save(modelfile)

In [12]:
model = gensim.models.Word2Vec.load(modelfile)

In [13]:
def remove_seen_and_uservec(predicted_list_w_scores, seen_list):
# predicted_list_w_scores is a list of lists,[[a,b]...] where a is ther user_id and b is the similairity score
    result_list = []
    for a, b in predicted_list_w_scores:
        if a not in seen_list and not a.startswith('u'):
            result_list.append([a, b])
    return result_list


def get_best_matches(userid):
    trained_wv_list = []
    for movie_id in train_dict[userid]:
        trained_wv_list.append(model.wv[movie_id])

    trained_wv_np = np.array(trained_wv_list)
    rec_list = []
    for mov_vec in trained_wv_list:
        temp_recommended_list = model.wv.most_similar(positive = [mov_vec],topn=TOPN_PERCLUSTER)
        temp_rec_seen_removed_list = remove_seen_and_uservec(temp_recommended_list, train_dict[userid])
        rec_list.extend(temp_rec_seen_removed_list)
    rec_list.sort(key = lambda element: element[1], reverse = True)  #rank based on similarity.  
    
# rec_list may have same movies with different scores.  Since it is sorted, in result_list we will only take the first score.

    rec_movieids_list = []
    rec_movies = 0
    rec_list_pointer = 0
    result_list = []
    while rec_movies < min(TOP_K * TOP_K_MULTIPLIER,len(rec_list)): # compute a larger list as input to the classifier; don't need it if not using a classifier
        if rec_list_pointer >= len(rec_list):
            break
        if rec_list[rec_list_pointer][0] not in rec_movieids_list:  #index 0 for movie id  
            rec_movieids_list.append(rec_list[rec_list_pointer][0])  #insert movie to rec_movieids_list
            element_list = [userid]
            element_list.extend(rec_list[rec_list_pointer])
            result_list.append(element_list)
            rec_movies = rec_movies + 1
        rec_list_pointer = rec_list_pointer + 1    
    return result_list

In [14]:
similarity_list = []

start_time = time.time()

for test_userid in test_dict:
#    print(test_userid,end=",")
    similarity_list.extend(get_best_matches(test_userid))
    
prediction_time = time.time() - start_time

print("\nTook {} seconds for prediction".format(prediction_time))


Took 12.106865882873535 seconds for prediction


In [15]:
predict_similarity_df = pd.DataFrame(similarity_list,columns=['userID','itemID','prediction'])
predict_similarity_df[['userID','itemID']] = predict_similarity_df[['userID','itemID']].astype(str)
predict_similarity_df.head()

,userID,itemID,prediction
0,1,588,0.983964
1,1,228,0.979110
2,1,418,0.969349
3,1,501,0.968286
4,1,5,0.967822


In [16]:
predict_similarity_file = EXP_NAME+'_DF.csv'
predict_similarity_df.to_csv(predict_similarity_file,index=False)

In [17]:
# p2 = pd.read_csv(predict_similarity_file)
# p2.head()

In [18]:
eval_map = map_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_ndcg = ndcg_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_precision = precision_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)
eval_recall = recall_at_k(test, predict_similarity_df, col_user='userID', col_item='itemID', col_rating='rating', k=TOP_K)

print("Word Embedding:")
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Word Embedding:
MAP:	0.046463
NDCG:	0.193828
Precision@K:	0.171474
Recall@K:	0.095319
